In [1]:
import pandas as pd
import psycopg2
#from psycopg2 import sql
#!ln -s /var/run/postgresql/.s.PGSQL.5432 /tmp/.s.PGSQL.5432'

In [2]:
conn = psycopg2.connect(database = "datamyneUSexports", user = "ram22")
cur = conn.cursor()

In [3]:
cur.execute("""
drop table if exists relevantbol;
create table relevantbol as
select *
from
(select *, duns_nbr_shipper::bigint as duns_number_bol from billoflading_all
WHERE duns_nbr_shipper != '' AND duns_nbr_shipper IS NOT NULL
) as bol
inner join 
(select shipper_d_b_duns_number::bigint as duns_number_db, 
shipper_d_b_duns_number,
shipper_d_b_business_name,
shipper_d_b_headquarter_duns_number, 
shipper_d_b_headquarter_parent_business_name,
shipper_d_b_domestic_ultimate_duns_number, 
shipper_d_b_domestic_ultimate_business_name,
shipper_d_b_global_ultimate_duns_number, 
shipper_d_b_global_ultimate_business_name
from children_db_profiles) as children
on (bol.duns_number_bol = children.duns_number_db);
""")
conn.commit()

In [4]:
cur.execute("""
COPY relevantbol to
    '/home/ram22/dataDrive/dataProjects/postgresql/tempOutput/relevantBoL.csv'
    WITH CSV HEADER;
""")
!cp '/home/ram22/dataDrive/dataProjects/postgresql/tempOutput/relevantBoL.csv' '../02_dataCreated/'

In [5]:
cur.execute("""
drop table if exists exp;
create table exp as
select *
from
(select *, duns_nbr_shipper::bigint as duns_number_bol from billoflading_all 
WHERE duns_nbr_shipper != '' AND duns_nbr_shipper IS NOT NULL
) as bol
left outer join 
(select shipper_d_b_duns_number::bigint as duns_number_db, 
shipper_d_b_duns_number,
shipper_d_b_business_name,
shipper_d_b_global_ultimate_duns_number, 
shipper_d_b_global_ultimate_business_name
from children_db_profiles) as children
on (bol.duns_number_bol = children.duns_number_db);
""")
conn.commit()

In [6]:
cur.execute("""
COPY exp to
    '/home/ram22/dataDrive/dataProjects/postgresql/tempOutput/exp.csv'
    WITH CSV HEADER;
""")
!cp '/home/ram22/dataDrive/dataProjects/postgresql/tempOutput/exp.csv' '../02_dataCreated/'
# Note: '790451231' is a Mitsubishi Forklift... in USA. It's global ultimate is '690539887', which is 
# MITSUBISHI NICHIYU FORKLIFT CO.,LTD. but that appears as it's own global ultimate. ie it never ties to our mitsu

In [7]:
conn.commit()
conn.close()

In [8]:
# Global ultimates which identify what we want
global_ultimates = [217304393, # BAE SYSTEMS PLC 
                    778557603, # CENTRICA PLC
                    790428754, # Chobani, LLC
                    305422255, # DONG Energy Power A/S
                    1315704, # E. I. Du Pont De Nemours and Company
                    12430880, # Herny Schein, Inc.
                    544957314, # Huawei Investment and Holding Co., Ltd.
                    671584550, # INDORAMA VENTURES PUBLIC COMPANY LIMITED
                    81466849, # Microsoft Corportation
                    690546460, # MITSUBISHI CORPORATION
                    480708874, # Nestle S.A.
                    6988414, # Newmont Mining Corporation
                    50957364, # Nike, Inc.
                    485609796, # Novartis AG
                    489840363, # Qiagen N.V.
                    144356508, # Qualcomm Incorporated
                    480008226, # Roche Holding AG
                    3296175, # The Coca-Cola Company
                    210300901, # UNILEVER PLC
                    289936783, # VODAFONE GROUP PLC
                    51957769, # Wal-Mart Stores, Inc.
                    387451045, # Louis Dreyfus Holding B.V.
                    79942718, # Alphabet, Inc.
                    60902413, # Google Inc.
                    461026429, # IBERDROLA, SOCIEDAD ANONIMA
                    238980408] #GLAXOSMITHKLINE PLC
domestic_ultimates = [366470602] # CONSOLIDATED CONTRACTORS INTERNATIONAL COMPANY S A L; 315275255 PUMA SE at this level only points at itself
headquarters = [315275255, 402491757, 794661801] # Puma SE; Heineken N.V.; Nespresso Usa, Inc.; 654292358 for Huawei Techonologies Co. Ltd.
entities = [862932619] # Bajaj Auto Corporation
# These make 31
print(len(global_ultimates) + len(domestic_ultimates) + len(headquarters) + len(entities))
# Not looked for: Klockner & Co, F. Hoffmann-La Roche Limited; Royal Philips; Reuters Market Light, India; Nykredit AS; Northwell Health; Dogan Broadcasting
# Not found: Mumm, Dignity Health , VTB Bank
# Problem with ROCHE
# Those are 11 (not double counting ROCHE). Plus 31 = 42. But I added Alphabet in global_ultimates.

31


In [9]:
relevantBoL = pd.read_csv('../02_dataCreated/relevantBoL.csv')

targetDBprofiles = pd.read_csv('../02_dataCreated/targetDBprofiles.csv')
targetDBprofiles.columns = ['shipper_d_b_' + i for i in targetDBprofiles.columns]

temp = relevantBoL.copy()

In [10]:
temp['shipper_d_b_business_name'][relevantBoL['shipper_d_b_global_ultimate_duns_number'].isin(global_ultimates)] = \
    relevantBoL['shipper_d_b_global_ultimate_business_name'][relevantBoL['shipper_d_b_global_ultimate_duns_number'].isin(global_ultimates)]
    
temp['shipper_d_b_business_name'][relevantBoL['shipper_d_b_domestic_ultimate_duns_number'].isin(domestic_ultimates)] = \
    relevantBoL['shipper_d_b_domestic_ultimate_business_name'][relevantBoL['shipper_d_b_domestic_ultimate_duns_number'].isin(domestic_ultimates)]
    
temp['shipper_d_b_business_name'][relevantBoL['shipper_d_b_headquarter_duns_number'].isin(headquarters)] = \
    relevantBoL['shipper_d_b_headquarter_parent_business_name'][relevantBoL['shipper_d_b_headquarter_duns_number'].isin(headquarters)]


/home/ram22/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ram22/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ram22/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 

In [11]:
temp['shipper_d_b_business_name'] = temp['shipper_d_b_business_name'].str.upper()
targetDBprofiles['shipper_d_b_business_name'] = targetDBprofiles['shipper_d_b_business_name'].str.upper()

In [12]:
temp = temp.merge(targetDBprofiles.drop(['shipper_d_b_duns_number',
    'shipper_d_b_headquarter_duns_number', 
    'shipper_d_b_headquarter_parent_business_name',
    'shipper_d_b_domestic_ultimate_duns_number', 
    'shipper_d_b_domestic_ultimate_business_name',
    'shipper_d_b_global_ultimate_duns_number', 
    'shipper_d_b_global_ultimate_business_name'], axis=1),
    how='left', 
    left_on='shipper_d_b_business_name', right_on='shipper_d_b_business_name',
    indicator=True)

In [13]:
##########################################################################
temp = temp[temp['shipper_d_b_business_name'] != str.upper('Nestl? S.A.')]
################################################################################

In [14]:
temp.to_csv('../02_dataCreated/newData.csv', index=False)

In [16]:
temp.shipper_d_b_business_name.value_counts(dropna=False)

MITSUBISHI CORPORATION                  3810
NESTLÉ S.A.                             3481
LOUIS DREYFUS HOLDING B.V.              1923
UNILEVER PLC                            1715
E. I. DU PONT DE NEMOURS AND COMPANY     770
THE COCA-COLA COMPANY                    445
WAL-MART STORES, INC.                    103
NIKE, INC.                                19
BAE SYSTEMS PLC                            6
HENRY SCHEIN, INC.                         2
MICROSOFT CORPORATION                      2
NEWMONT MINING CORPORATION                 1
Name: shipper_d_b_business_name, dtype: int64

In [17]:
temp.shipper_d_b_foundIn.value_counts()

global ultimate    10354
Name: shipper_d_b_foundIn, dtype: int64

In [76]:
temp.shape

(93929, 217)

In [77]:
argh = temp.groupby(['consignee_d_b_business_name', 'consignee_declared'])

In [78]:
argh['consignee_declared'].count().to_csv('counts430.csv')